In [ ]:
# done done


In [1]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'belfercenter_org_publication'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# creating pdf directories
pdf_directory = c.create_pdf_directories(project_path, site)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_url = 0
unable_to_fetch_page_url = 0
unable_to_download_pdf = 0
publish_source = 'belfercenter.org'
country = 'United States'
language = 'English'
images_path = []

news_page = c.download_page('https://www.belfercenter.org/rss-research')

for _ in news_page.split('class="row-item">')[1:]:
    
    rss_url = c.scrap('href="(.*?)"', _)
    if 'http' not in rss_url:
        rss_url = 'https://www.belfercenter.org' + rss_url

    rss = c.download_page(rss_url)
    logger.info(f'Fetching rss {rss_url}\n')
    if rss.startswith('Unable to fetch'):
            logger.info(rss) # writes error message with error code
            unable_to_fetch_page_url += 1
            continue    

    for i in rss.split('<item>')[1:]:

        # source_link
        source_link = c.scrap("<link>(.*?)<", i)

        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        time.sleep(random.randint(1,3))
        
        # source_headline
        source_headline = c.scrap('<title>(.*?)<', i)
        source_headline = re.sub('&#x27;|&quot;', "'", source_headline, flags=re.S)

        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            # print(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue
        
        page = c.download_page(source_link)
        if page.startswith('Unable to fetch'):
                logger.info(page) # writes error message with error code
                unable_to_fetch_url += 1
                continue        
        
        # Date and time
        pub_date, publish_time = '', ''
        if rss_url == 'https://www.belfercenter.org/rss/round-up/73986/feed':
                date_time_str = c.scrap('<pubDate>(.*?)\s*\+', i) #Thu, 01 Dec 2016 00:20:59
                date_time_obj = datetime.strptime(date_time_str, '%a, %d %b %Y %H:%M:%S') #2021-02-08T04:00:00
                ist_date_time = date_time_obj + timedelta(hours = 5,minutes = 30)  # utc time to ist
                ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
                pub_date = ist_date_time[:10]
                publish_time = ist_date_time[11:]
        
        else:
            try:
                date_time_str = c.scrap('hks:publication_effective_date"\s*content="(.*?)Z"', page)
                date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%dT%H:%M:%S') #2021-02-08T04:00:00
                ist_date_time = date_time_obj + timedelta(hours = 5,minutes = 30)  # utc time to ist
                ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
                pub_date = ist_date_time[:10]
                publish_time = ist_date_time[11:]
            except:
                pass
       # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')
            # print(f'Skipping due to date {source_link}\n')
            skipped_due_to_date += 1
            continue

        # break if date is not today's date
        if pub_date != created_on:
            break    

        logger.info(f'Fetching {source_link}\n')
        # print(f'Fetching {source_link}\n')

        # source_content          

        source_content = c.scrap('<h2\s*class="subtitle">(.*?)<div\s*class="field\s*field--name-field-full', page)
        if not source_content:
            source_content = c.scrap('id=\'field-page-content\'>(.*?)<div\s*class="more-information">', page)            
        source_content = re.sub('<ul\s*class="actions".*?</ul>|<h2\s*class="header-text">.*', '', source_content, flags=re.S)
        source_content = re.sub('&ndash;|&mdash;', '-', source_content, flags=re.S)
        source_content = c.strip_html(source_content)
 
        
        # skip if content not found
        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')
            # print(f'Skipping due to content {source_link}\n')
            skipped_due_to_content += 1
            continue

        # journalist
        journalist = c.scrap('<div\s*class="view-publication-author-and-date">\s*(.*?)</ul>', page)
        journalist = re.sub('<span.*?</span>', "", journalist, flags=re.S)
        journalist = re.sub('\n\s*', ' ', journalist, flags=re.S)
        journalist = c.strip_html(journalist)

        if not journalist:
            journalist = 'NA'
        
        # current date and time 
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []
        
        pdf_path, pdf_name = '', ''
        reg = ('class="file-item".*?href="(.*?)"','class="button-item".*?href="(.*?)"')
        
        for p in reg:
            pdf_url = c.scrap(p, page)
            if 'http' not in pdf_url:
                pdf_url = 'https://www.belfercenter.org' + pdf_url
            if pdf_url:
                break

        if pdf_url:
            if 'pdf' in pdf_url:
                # pdf_name
                pdf_name = c.scrap('.*/(.*)', pdf_url)
                pdf_name = pdf_name.replace('%20','_')
            
                # pdf_path
                pdf_path = f'{pdf_directory}/{pdf_name}'        
            
                # download pdf
                pdf = c.download_pdf(pdf_url, pdf_path)
                if pdf.startswith('Unable to fetch'):
                    logger.info(pdf) # writes error message with error code
                    unable_to_download_pdf += 1
                    continue
           
        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":'',
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content,
                        "pdf_url": pdf_url,
                        "pdf_name": pdf_name,
                        "pdf_path":pdf_path
                    }

#         cl_data.insert_one(clientdata)  # get object id and insert data
        no_of_data += 1

logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch url: {unable_to_fetch_url}\n')
logger.info(f'Unable to fetch url: {unable_to_fetch_page_url}\n')
logger.info(f'Unable to download pdf: {unable_to_download_pdf}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')
